In [8]:
#!pip install python-twitter
import twitter
#from __future__ import print_function
import json
import sys
import pandas as pd

In [9]:

def get_tweets(api=None, screen_name=None):
    timeline = api.GetUserTimeline(screen_name=screen_name, count=200,exclude_replies=True)
    earliest_tweet = min(timeline, key=lambda x: x.id).id
    print("getting tweets before:", earliest_tweet)

    while True:
        tweets = api.GetUserTimeline(
            screen_name=screen_name, max_id=earliest_tweet, count=200
        )
        new_earliest = min(tweets, key=lambda x: x.id).id

        if not tweets or new_earliest == earliest_tweet:
            break
        else:
            earliest_tweet = new_earliest
            print("getting tweets before:", earliest_tweet)
            timeline += tweets

    return timeline

In [24]:

api = twitter.Api(consumer_key='AhsxA4rErQMya8vd5Qx3f79N5',
                  consumer_secret='3SmPMO9Aw4MMrUh950jCxqLj8kn1AOVKbKwuBwlR2T5LWDCMe6',
                  access_token_key='1105103979770462210-Ggl91DoyVuggOo8FRXU68TI39o5uEi',
                  access_token_secret='5ZaoKCEh4xOBGUctZInKzEOZhWNMSfKPT8pbwVkFSTpnV')

#----------------------------------------------------------------------
screen_name = 'Metrovias' #<<<<<<<<<---------------------INPUT---------
#----------------------------------------------------------------------


#get tweets
timeline = get_tweets(api=api, screen_name=screen_name)

#saves json
with open(screen_name + '.json', 'w+') as f:
    for tweet in timeline:
        f.write(json.dumps(tweet._json))
        f.write('\n')
        
#open json creates DF
with open(screen_name + '.json', 'r') as f:
    for line in f:
        tweet = json.loads(line)
        #ts = time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(tweet['created_at'],'%a %b %d %H:%M:%S +0000 %Y'))
        twtsdate.append(tweet['created_at'])
        twtstxt.append(tweet['text'])

twtsdates = pd.Series(twtsdate)
twtstxts = pd.Series(twtstxt)

df = pd.concat([twtsdates.rename('DATE') ,twtstxts.rename("TEXT")] ,  axis=1)
df['DATE'] = pd.to_datetime(pd.Series(df['DATE']))

getting tweets before: 1094511776430022656
getting tweets before: 1084828041346465793
getting tweets before: 1074742026460168192
getting tweets before: 1067366829071523840
getting tweets before: 1060501447278780417
getting tweets before: 1054342522200342533
getting tweets before: 1047461700478406656
getting tweets before: 1041277011115819008
getting tweets before: 1034493287145656323
getting tweets before: 1026739992645783552
getting tweets before: 1020085631555338241
getting tweets before: 1012677604187484160
getting tweets before: 1004801653143490562
getting tweets before: 998153886723604480
getting tweets before: 992751084476170241
getting tweets before: 987668535106252800
getting tweets before: 987303537070804993


In [27]:

df

,DATE,TEXT
0,2019-03-12 13:28:44,#LíneaD - Ya realiza su recorrido completo ent...
1,2019-03-12 13:14:44,#LíneaD - Circula con servicio limitado entre ...
2,2019-03-12 13:02:43,#LíneaD - Servicio limitado entre las estacion...
3,2019-03-12 12:13:42,#LíneaD - Circula con demora. 09:13
4,2019-03-12 12:09:41,#LíneaD - Servicio con demora. 09:09
5,2019-03-11 20:36:37,#LíneaD - Ya circula con su frecuencia normal....
6,2019-03-11 20:34:36,#LíneaD - Servicio con demora. 17:34
7,2019-03-11 20:31:35,#LíneaD - Ya realiza su recorrido completo ent...
8,2019-03-11 18:54:34,#LíneaD - Servicio limitado entre las estacion...
9,2019-03-11 16:37:31,#LíneaB - Ya circula con su frecuencia normal....


In [39]:
import requests

#http://docs.python-requests.org/en/latest/user/quickstart/#make-a-request
ci = "01c8737dd3da4c67a5d822f9e174f20c"
cis= "bA0d80a2dFd544709e7632CC08442c55"
resp = requests.get('https://apitransporte.buenosaires.gob.ar/subtes/feed-gtfs?client_id=' + ci + '&client_secret=' + cis)


data = resp.content


#f = open('test.zip', 'wb')
#f.write(data.encode('utf8'))
#f.close()

from zipfile import ZipFile
#import cStringIO as StringIO
import StringIO

in_memory = StringIO.StringIO(data)
zf = ZipFile(in_memory, "a")  

zf.writestr("file.txt", "some text contents")

zf.close()

new_zip_data = in_memory.getvalue()

ModuleNotFoundError: No module named 'StringIO'